In [1]:
# TODO: clean up imports

import numpy as np
import torch
import math
import os
import random
from torch import nn
import torchvision
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import ToTensor, Lambda, Compose
from torch.optim.lr_scheduler import MultiStepLR

In [2]:
# TODO: check parameters, code >>> paper 

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 128 # TODO: 95% right
test_batch_size = 10 # TODO: probably bigger, doesn't really impact performance
classes_group = 10 # TODO: check with 2, 5 and 50 as well
epochs = 70 # TODO: should be 60, 200 doesn't change performance meaningfully
_lr = 2 #0.1 # TODO: 2 is way too high but iCarl uses it, what's in the code?
wd = 1e-4 # TODO: 99% right #1
mom = 0.9 # TODO: 99.99% right
pic_height = 32 # TODO: probably not needed

In [3]:
# Once we have finished repository
'''
if not os.path.isdir('./...'):
  !git clone https://github.com/...

DATA_DIR = './...'
from ... .data import Cifar100
'''

# Load the dataset with custom class
from data import Cifar100
DATA_DIR = '.'
training_data = Cifar100(DATA_DIR, True, transform=transforms.ToTensor())
test_data = Cifar100(DATA_DIR, False, transform=transforms.ToTensor())

print(f'Train Dataset: {len(training_data)}')
print(f'Test Dataset: {len(test_data)}')

Files already downloaded and verified
Files already downloaded and verified
Train Dataset: 50000
Test Dataset: 10000


In [4]:
# TODO: Test vs resnet32 (should be comparable: source iCarl, pag. 6) (use _resnet() (careful there's an underscore!) from pytorch, just need  to find the right list for "layers" argument)

from resnet_cifar import resnet32
model = resnet32(num_classes=classes_group)
model = model.to(device)
loss_fn = nn.BCEWithLogitsLoss() #BCE + sogmoid all in one
optimizer = torch.optim.SGD(model.parameters(), lr=_lr, weight_decay=wd, momentum = mom)

In [5]:
train_batches, test_batches = [], []
for i in range(10):
  train_batch_indexes = training_data.__BatchIndexes__(i)
  
  test_batch_indexes = []
  for j in range(i+1):
    test_indexes = test_data.__BatchIndexes__(j)
    test_batch_indexes += test_indexes

  train_batch = Subset(training_data,train_batch_indexes)
  test_batch = Subset(test_data,test_batch_indexes)

  train_batches.append(train_batch)
  test_batches.append(test_batch)

In [6]:
# TODO: clean up methods, should probably implement eval method in utils.py or similar, see below

train_loader = DataLoader(train_batches[0], batch_size, True, drop_last=True)
from data import augmentate 
model.train()
torch.set_printoptions(edgeitems=5) # just for debug purpouses

# TODO: check scheduler parameters, is it even needed?
scheduler = MultiStepLR(optimizer, milestones=[48, 62], gamma=0.2)
for epoch in range(epochs):
  model.train()
  print(f"Epoch: {epoch+1:02} ", end="")
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(train_loader, 0): # batch is just the index
  #for X, y in train_loader
    X, y = X.to(device), torch.from_numpy(np.array(y)).to(device)
    # Data augmentation
    X = torch.stack([augmentate(image) for image in X])
    #X = torch.stack([image for image in X ])
    optimizer.zero_grad() # needs to be before predictions
 
    # Compute prediction loss
    outputs = model(X) #pred.size([128, 10])
    pred = torch.argmax(outputs.data, 1)
    oh_y = torch.zeros(outputs.shape[0], outputs.shape[1])
    oh_y[range(oh_y.shape[0]), y.long()]=1
    loss = loss_fn(outputs, oh_y.to(device)) 

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update loss
    train_loss += loss

  print(f"loss: {train_loss/batch:.4f}, ", end = "")
  scheduler.step()

  # eval per epoch
  # TODO: probably needs to move to a utils.py 
  with torch.no_grad():

    correct = 0
    total = 0

    model.eval()
    # TODO: be consistant with the way of handling dataloaders (unpack inside of for cycle or during for inizialization)
    for data in train_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        wrn_outputs = model(images)
        outputs = wrn_outputs
        predicted = torch.argmax(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_accuracy = correct / total
    epoch_accuracy = round(100 * epoch_accuracy, 4)

  print(f"acc: {epoch_accuracy:.2f}%")

print('Finished Training')

Epoch: 01 loss: 0.4517, acc: 10.96%
Epoch: 02 loss: 0.3321, acc: 21.77%
Epoch: 03 loss: 0.3216, acc: 21.89%
Epoch: 04 loss: 0.3105, acc: 11.52%
Epoch: 05 loss: 0.3011, acc: 26.18%
Epoch: 06 loss: 0.2882, acc: 29.59%
Epoch: 07 loss: 0.2910, acc: 32.43%
Epoch: 08 loss: 0.2857, acc: 18.65%
Epoch: 09 loss: 0.2830, acc: 38.98%
Epoch: 10 loss: 0.2789, acc: 15.99%
Epoch: 11 loss: 0.2797, acc: 27.74%
Epoch: 12 loss: 0.2740, acc: 43.43%
Epoch: 13 loss: 0.2766, acc: 26.28%
Epoch: 14 loss: 0.2694, acc: 28.61%
Epoch: 15 loss: 0.2677, acc: 12.84%
Epoch: 16 loss: 0.2698, acc: 20.21%
Epoch: 17 loss: 0.2642, acc: 27.88%
Epoch: 18 loss: 0.2585, acc: 29.99%
Epoch: 19 loss: 0.2538, acc: 11.26%
Epoch: 20 loss: 0.2530, acc: 26.92%
Epoch: 21 loss: 0.2459, acc: 12.78%
Epoch: 22 loss: 0.2501, acc: 38.72%
Epoch: 23 loss: 0.2474, acc: 12.10%
Epoch: 24 loss: 0.2472, acc: 12.06%
Epoch: 25 loss: 0.2501, acc: 40.28%
Epoch: 26 loss: 0.2474, acc: 25.96%
Epoch: 27 loss: 0.2455, acc: 40.34%
Epoch: 28 loss: 0.2370, acc:

In [7]:
# TODO: evaluate_model should be moved to a "test_model" and in utils.py, needs to be cleaned up. Use a single way of computing accuracy, needs to be consistent with eval_per_epoch in training

def evaluate_model(test_dl, model):
  model.eval()
  correct, wrong = 0, 0
  # TODO: check device, clean variables
  tot_pred, ground_truth, fin_probs,tot_pred2, = list(), list(), list(), list()
  for i, (inputs, targets) in enumerate(test_dl):
    inputs, targets = inputs.to(device), targets.to('cpu')
    ground_truth.extend(targets.numpy().tolist())
    with torch.no_grad():
      pred = model(inputs)
      pred = pred.cpu()
      pred = torch.argmax(pred.data, 1)
      tot_pred.extend(*pred.reshape((1,pred.shape[0])).numpy().tolist())
      for index, p in enumerate(pred):
        if p == targets[index]:
          correct += 1
        else:
          wrong += 1

  # TODO: improve outputs
  acc = correct/(correct+wrong)
  print(f'sk2: {accuracy_score(ground_truth, tot_pred)}')
  return acc


In [8]:
test_loader = DataLoader(test_batches[0], batch_size, True)
evaluate_model(test_loader, model)

sk2: 0.806


0.806